In [ ]:
import numpy as np
import json
import nltk
import tensorflow as tf

In [3]:

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:


# Load the training and testing data
with open('intents.json') as f:
    intents = json.load(f)

with open('intents_testing.json') as f:
    testing_intents = json.load(f)

# Preprocess the data
words = []
labels = []
docs_x = []
docs_y = []

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the pattern
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

# Convert the words and labels to unique arrays
words = sorted(set(words))
labels = sorted(set(labels))

# Create training and testing data sets
training = []
output = []
out_empty = [0] * len(labels)

for i, doc in enumerate(docs_x):
    bag = []

    for word in words:
        if word in doc:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[i])] = 1

    training.append(bag)
    output.append(output_row)

training = np.array(training)
output = np.array(output)


In [4]:

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(words), 64),
    tf.keras.layers.GRU(64, dropout=0.1),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(labels), activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(training, output, epochs=100, batch_size=8)

# Evaluate the model on the testing data
for testing_intent in testing_intents['intents']:
    for testing_pattern in testing_intent['patterns']:
        testing_bag = []
        testing_wrds = nltk.word_tokenize(testing_pattern)
        for w in words:
            testing_bag.append(1) if w in testing_wrds else testing_bag.append(0)

        testing_bag = np.array(testing_bag)
        testing_bag = testing_bag.reshape(1, -1)
        result = model.predict(testing_bag)
        tag = labels[np.argmax(result)]


Epoch 1/100
109/109 [==============================] - 93s 819ms/step - loss: 3.6140 - accuracy: 0.0746
Epoch 2/100
109/109 [==============================] - 80s 737ms/step - loss: 3.5057 - accuracy: 0.1079
Epoch 3/100
109/109 [==============================] - 81s 745ms/step - loss: 3.4774 - accuracy: 0.1079
Epoch 4/100
109/109 [==============================] - 80s 737ms/step - loss: 3.4825 - accuracy: 0.1079
Epoch 5/100
109/109 [==============================] - 82s 752ms/step - loss: 3.4727 - accuracy: 0.1079
Epoch 6/100
109/109 [==============================] - 83s 762ms/step - loss: 3.4748 - accuracy: 0.1079
Epoch 7/100
109/109 [==============================] - 81s 741ms/step - loss: 3.4760 - accuracy: 0.1079
Epoch 8/100
109/109 [==============================] - 81s 743ms/step - loss: 3.4698 - accuracy: 0.1079
Epoch 9/100
109/109 [==============================] - 81s 747ms/step - loss: 3.4730 - accuracy: 0.1079
Epoch 10/100
109/109 [==============================] - 81s 744m

In [5]:
num_correct = 0
total = 0

for testing_intent in testing_intents['intents']:
    for testing_pattern in testing_intent['patterns']:
        testing_bag = []
        testing_wrds = nltk.word_tokenize(testing_pattern)
        for w in words:
            testing_bag.append(1) if w in testing_wrds else testing_bag.append(0)

        testing_bag = np.array(testing_bag)
        testing_bag = testing_bag.reshape(1, -1)
        result = model.predict(testing_bag)
        tag = labels[np.argmax(result)]

        if tag == testing_intent['tag']:
            num_correct += 1

        total += 1

accuracy = num_correct / total
print(f"Accuracy: {accuracy*100:.2f}%")


1/1 [==============================] - 0s 71ms/step
Accuracy: 10.53%
